# <span style="color:#4e5ec5">Import Packages</span>


In [189]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
import calendar
from math import sin, cos, sqrt, atan2, radians,asin
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap
from folium.plugins import TimestampedGeoJson
from folium.plugins import MarkerCluster
from geopy.distance import great_circle
import matplotlib.dates as mdates
import matplotlib as mpl
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb 

# <span style="color:#4e5ec5">Load Train and Test Datasets</span>


In [190]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834,56,0.0000,64,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791,47,0.0000,134,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087,80,0.0000,61,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598,271,15.6638,68,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189129552,10.5,3407,182,0.0000,112,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct


In [191]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16967 entries, 0 to 16966
Data columns (total 14 columns):
tripid                       16967 non-null int64
additional_fare              16967 non-null float64
duration                     16967 non-null int64
meter_waiting                16967 non-null int64
meter_waiting_fare           16967 non-null float64
meter_waiting_till_pickup    16967 non-null int64
pickup_time                  16967 non-null object
drop_time                    16967 non-null object
pick_lat                     16967 non-null float64
pick_lon                     16967 non-null float64
drop_lat                     16967 non-null float64
drop_lon                     16967 non-null float64
fare                         16967 non-null float64
label                        16967 non-null object
dtypes: float64(7), int64(4), object(3)
memory usage: 1.8+ MB


# <span style="color:#4e5ec5">Pre process pickup and drop off times</span>

### <span style="color:#4ec57a"> Convert pickup and drop off time columns to date type. Split the Pickup time and Drop time in to year-month-day-day of week-hour</span>

In [192]:
train['pickup_time']=pd.to_datetime(train['pickup_time'])
train['drop_time']=pd.to_datetime(train['drop_time'])
train.head()

test['pickup_time']=pd.to_datetime(test['pickup_time'])
test['drop_time']=pd.to_datetime(test['drop_time'])

In [193]:
# beak the pickup date time
train['pickup_date']= train['pickup_time'].dt.date
train['pickup_day']=train['pickup_time'].apply(lambda x:x.day)
train['pickup_hour']=train['pickup_time'].apply(lambda x:x.hour)
train['pickup_day_of_week']=train['pickup_time'].apply(lambda x:x.dayofweek)
train['pickup_month']=train['pickup_time'].apply(lambda x:x.month)
train['pickup_year']=train['pickup_time'].apply(lambda x:x.year)

test['pickup_date']= test['pickup_time'].dt.date
test['pickup_day']=test['pickup_time'].apply(lambda x:x.day)
test['pickup_hour']=test['pickup_time'].apply(lambda x:x.hour)
test['pickup_day_of_week']=test['pickup_time'].apply(lambda x:x.dayofweek)
test['pickup_month']=test['pickup_time'].apply(lambda x:x.month)
test['pickup_year']=test['pickup_time'].apply(lambda x:x.year)

# beak the drop off date time
train['drop_date']= train['drop_time'].dt.date
train['drop_day']=train['drop_time'].apply(lambda x:x.day)
train['drop_hour']=train['drop_time'].apply(lambda x:x.hour)
train['drop_day_of_week']=train['drop_time'].apply(lambda x:x.dayofweek)
train['drop_month']=train['drop_time'].apply(lambda x:x.month)
train['drop_year']=train['drop_time'].apply(lambda x:x.year)

test['drop_date']= test['drop_time'].dt.date
test['drop_day']=test['drop_time'].apply(lambda x:x.day)
test['drop_hour']=test['drop_time'].apply(lambda x:x.hour)
test['drop_day_of_week']=test['drop_time'].apply(lambda x:x.dayofweek)
test['drop_month']=test['drop_time'].apply(lambda x:x.month)
test['drop_year']=test['drop_time'].apply(lambda x:x.year)

train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,0,4,11,2019,2019-11-01,1,0,4,11,2019
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,0,4,11,2019,2019-11-01,1,1,4,11,2019
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,1,4,11,2019,2019-11-01,1,1,4,11,2019
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,2,4,11,2019,2019-11-01,1,2,4,11,2019
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,5,4,11,2019,2019-11-01,1,6,4,11,2019


### <span style="color:#4ec57a">New column to hold label as boolean value</span>

In [194]:
train['label_bool']=train['label'].apply(lambda x:x=='correct')

train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,...,pickup_day_of_week,pickup_month,pickup_year,drop_date,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,label_bool
0,189123628,10.5,834,56,0.0000,64,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,...,4,11,2019,2019-11-01,1,0,4,11,2019,True
1,189125358,10.5,791,47,0.0000,134,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
2,189125719,10.5,1087,80,0.0000,61,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,...,4,11,2019,2019-11-01,1,1,4,11,2019,True
3,189127273,10.5,598,271,15.6638,68,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,...,4,11,2019,2019-11-01,1,2,4,11,2019,True
4,189129552,10.5,3407,182,0.0000,112,2019-11-01 05:38:00,2019-11-01 06:35:00,7.13402,79.8969,...,4,11,2019,2019-11-01,1,6,4,11,2019,True


### check for missing values

In [195]:
train[pd.isnull(train)].sum()
test[pd.isnull(test)].sum()

tripid                       0.0
additional_fare              0.0
duration                     0.0
meter_waiting                0.0
meter_waiting_fare           0.0
meter_waiting_till_pickup    0.0
pickup_time                  0.0
drop_time                    0.0
pick_lat                     0.0
pick_lon                     0.0
drop_lat                     0.0
drop_lon                     0.0
fare                         0.0
pickup_date                  0.0
pickup_day                   0.0
pickup_hour                  0.0
pickup_day_of_week           0.0
pickup_month                 0.0
pickup_year                  0.0
drop_date                    0.0
drop_day                     0.0
drop_hour                    0.0
drop_day_of_week             0.0
drop_month                   0.0
drop_year                    0.0
dtype: float64

In [196]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year
count,1.696700e+04,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,...,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000
mean,2.014702e+08,13.721627,1702.918901,628.909118,32.046838,112.490364,6.965130,79.997982,6.965852,79.999153,...,15.859433,13.803029,2.919785,7.949844,2019.339836,15.860376,14.082277,2.921730,7.948665,2019.339954
std,7.453342e+06,98.995408,8121.788501,8064.894289,465.577907,442.142959,0.210811,0.242792,0.211475,0.242978,...,8.843951,4.819491,1.940130,5.002983,0.473667,8.844053,4.881715,1.940802,5.003443,0.473707
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,1.000000,0.000000,0.000000,1.000000,2019.000000,1.000000,0.000000,0.000000,1.000000,2019.000000
25%,1.947352e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864830,79.870450,6.865580,79.870100,...,8.000000,10.000000,1.000000,1.000000,2019.000000,8.000000,11.000000,1.000000,1.000000,2019.000000
50%,2.013933e+08,10.500000,1009.000000,123.000000,0.115600,73.000000,6.913970,79.898600,6.915930,79.899000,...,16.000000,14.000000,3.000000,11.000000,2019.000000,16.000000,14.000000,3.000000,11.000000,2019.000000
75%,2.083317e+08,10.500000,1802.500000,351.000000,14.334400,144.000000,7.042645,79.956450,7.046540,79.960000,...,23.000000,18.000000,5.000000,12.000000,2020.000000,23.000000,18.000000,5.000000,12.000000,2020.000000
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,8.064930,81.009700,...,31.000000,23.000000,6.000000,12.000000,2020.000000,31.000000,23.000000,6.000000,12.000000,2020.000000


### <span style="color:#4ec57a">Function created to calculate distance based on longitudes and latitudes of pickup and drop off locations</span>

In [197]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [198]:
haversine_distance('pick_lat', 'pick_lon', 'drop_lat', 'drop_lon')
train['H_Distance'].head(10)

0    5.092770 
1    3.168058 
2    6.305395 
3    0.861946 
4    24.207039
5    4.777624 
6    5.322544 
7    1.035302 
8    2.930715 
9    14.381001
Name: H_Distance, dtype: float64

In [199]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,...,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,drop_day,drop_hour,drop_day_of_week,drop_month,drop_year,H_Distance
count,1.696700e+04,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,...,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000,16967.000000
mean,2.014702e+08,13.721627,1702.918901,628.909118,32.046838,112.490364,6.965130,79.997982,6.965852,79.999153,...,13.803029,2.919785,7.949844,2019.339836,15.860376,14.082277,2.921730,7.948665,2019.339954,4.524499
std,7.453342e+06,98.995408,8121.788501,8064.894289,465.577907,442.142959,0.210811,0.242792,0.211475,0.242978,...,4.819491,1.940130,5.002983,0.473667,8.844053,4.881715,1.940802,5.003443,0.473707,4.602826
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,79.796700,...,0.000000,0.000000,1.000000,2019.000000,1.000000,0.000000,0.000000,1.000000,2019.000000,0.000000
25%,1.947352e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864830,79.870450,6.865580,79.870100,...,10.000000,1.000000,1.000000,2019.000000,8.000000,11.000000,1.000000,1.000000,2019.000000,1.486029
50%,2.013933e+08,10.500000,1009.000000,123.000000,0.115600,73.000000,6.913970,79.898600,6.915930,79.899000,...,14.000000,3.000000,11.000000,2019.000000,16.000000,14.000000,3.000000,11.000000,2019.000000,3.216324
75%,2.083317e+08,10.500000,1802.500000,351.000000,14.334400,144.000000,7.042645,79.956450,7.046540,79.960000,...,18.000000,5.000000,12.000000,2020.000000,23.000000,18.000000,5.000000,12.000000,2020.000000,6.150903
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,8.064930,81.009700,...,23.000000,6.000000,12.000000,2020.000000,31.000000,23.000000,6.000000,12.000000,2020.000000,79.220774


In [200]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'drop_date',
       'drop_day', 'drop_hour', 'drop_day_of_week', 'drop_month', 'drop_year',
       'label_bool', 'H_Distance'],
      dtype='object')

In [201]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'pickup_date', 'pickup_day', 'pickup_hour', 'pickup_day_of_week',
       'pickup_month', 'pickup_year', 'drop_date', 'drop_day', 'drop_hour',
       'drop_day_of_week', 'drop_month', 'drop_year', 'H_Distance'],
      dtype='object')

# <span style="color:#c54e5e">Method 1: Classification Using Random Forests</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

### <span style="color:#4ec57a">Copy of cleaned data set

In [202]:
C_train = train.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date', 'label'], axis = 1)
C_test = test.drop(['tripid','pickup_time', 'drop_time', 'pickup_date', 'drop_date'], axis = 1)

### <span style="color:#4ec57a">Prepare train and test data sets

In [203]:
x_train = C_train.iloc[:,C_train.columns!='label_bool']
y_train = C_train['label_bool'].values
x_test = C_test

### <span style="color:#4ec57a">Random Forest Model and Prediction

In [204]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)

### <span style="color:#4ec57a">Prepare for submission

In [205]:
sample_submission_random_forest = test.iloc[:, lambda df: [0]]
sample_submission_random_forest['label_bool'] = rf_predict

sample_submission_random_forest['prediction']=sample_submission_random_forest['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_random_forest = sample_submission_random_forest.drop(['label_bool'], axis = 1)
sample_submission_random_forest.to_csv('sample_submission_random_forest.csv', index=False)

sample_submission_random_forest.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


# <span style="color:#c54e5e">Method 2: Classification Using LIGHTGBM</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>


In [236]:
lightgbm_params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 80,
        'learning_rate': 0.05,
        'max_depth': 7,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1     
    }

In [237]:
pred_test_y = np.zeros(x_test.shape[0])

In [238]:
train_set = lgbm.Dataset(x_train, y_train, silent=True)

In [239]:
lgbm_model = lgbm.train(lightgbm_params, train_set = train_set, num_boost_round=3000)

In [240]:
pred_test_y = lgbm_model.predict(x_test, num_iteration = lgbm_model.best_iteration)

In [242]:
sample_submission_lgbm = test.iloc[:, lambda df: [0]]
sample_submission_lgbm['label_bool'] = pred_test_y
sample_submission_lgbm.head(20)

sample_submission_lgbm['prediction']=sample_submission_lgbm['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_lgbm = sample_submission_lgbm.drop(['label_bool'], axis = 1)
sample_submission_lgbm.to_csv('sample_submission_lgbm.csv', index=False)

sample_submission_lgbm.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


# <span style="color:#c54e5e">Method 3: Classification Using XGBM</span>

##### <span style="color:#4ec57a">In this method I'm trying to predict the correctnes value which will be between 0 and 1. Prediction will be evaluated based on the value >= 0.5 or not</span>

In [218]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test)

In [225]:
xgbm_params = {'max_depth':20,
          'eta':1,
          'silent':1,
          'objective':'reg:squarederror',
          'eval_metric':'rmse',
          'learning_rate':0.05
         }
xgbm_num_rounds = 500

In [226]:
xb = xgb.train(xgbm_params, dtrain, xgbm_num_rounds)


[18:23:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [227]:
y_pred_xgb = xb.predict(dtest)

In [229]:
sample_submission_xgbm = test.iloc[:, lambda df: [0]]
sample_submission_xgbm['label_bool'] = pred_test_y
sample_submission_xgbm.head(20)

sample_submission_xgbm['prediction']=sample_submission_xgbm['label_bool'].apply(lambda x:1 if x >= 0.5 else 0)

sample_submission_xgbm = sample_submission_xgbm.drop(['label_bool'], axis = 1)
sample_submission_xgbm.to_csv('sample_submission_xgbm.csv', index=False)

sample_submission_xgbm.head(5)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
